In [3]:
import pandas as pd
import numpy as np

events = pd.read_csv("data/processed/events_ucl_2018_19.csv")

In [15]:
events[['x', 'y']] = (
    events['location']
    .dropna()
    .apply(lambda loc: pd.Series(loc))
)

ValueError: Columns must be same length as key

In [ ]:
events[['x', 'y']].describe()

KeyError: "None of [Index(['x', 'y'], dtype='str')] are in the [columns]"

In [5]:
X_BINS = np.linspace(0, 120, 7)  # 6 vertical zones
Y_BINS = np.linspace(0, 80, 6)   # 5 horizontal zones

In [6]:
events['x_zone'] = pd.cut(events['x'], bins=X_BINS, labels=False, include_lowest=True)
events['y_zone'] = pd.cut(events['y'], bins=Y_BINS, labels=False, include_lowest=True)

events['zone_id'] = events['y_zone'] * 6 + events['x_zone']

KeyError: 'x'

In [7]:
SPATIAL_EVENTS = [
    'Pass',
    'Carry',
    'Shot',
    'Pressure',
    'Ball Recovery'
]

spatial_events = events[events['type'].isin(SPATIAL_EVENTS)].copy()

In [8]:
zone_counts = (
    spatial_events
    .groupby(['match_id', 'team', 'zone_id'])
    .size()
    .reset_index(name='count')
)

KeyError: 'zone_id'

In [9]:
zone_totals = (
    zone_counts
    .groupby(['match_id', 'team'])['count']
    .sum()
    .reset_index(name='total')
)

zone_features = zone_counts.merge(
    zone_totals,
    on=['match_id', 'team']
)

zone_features['zone_pct'] = (
    zone_features['count'] / zone_features['total']
)

NameError: name 'zone_counts' is not defined

In [10]:
spatial_vector = (
    zone_features
    .pivot_table(
        index=['match_id', 'team'],
        columns='zone_id',
        values='zone_pct',
        fill_value=0
    )
    .reset_index()
)

NameError: name 'zone_features' is not defined

In [ ]:
spatial_vector.columns = [
    f'zone_{c}' if isinstance(c, int) else c
    for c in spatial_vector.columns
]

In [11]:
avg_height = (
    spatial_events
    .groupby(['match_id', 'team'])['x']
    .mean()
    .reset_index(name='avg_action_height')
)

KeyError: 'Column not found: x'

In [12]:
avg_width = (
    spatial_events
    .groupby(['match_id', 'team'])['y']
    .std()
    .reset_index(name='width_spread')
)

KeyError: 'Column not found: y'

In [13]:
spatial_features = spatial_vector.merge(
    avg_height, on=['match_id', 'team'], how='left'
).merge(
    avg_width, on=['match_id', 'team'], how='left'
)

NameError: name 'spatial_vector' is not defined

In [14]:
spatial_features.describe()

NameError: name 'spatial_features' is not defined